<a href="https://colab.research.google.com/github/hudengjunai/Colab_notebooks/blob/master/model_pruning_with_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip list|grep tensorflow
import tensorflow as tf

mesh-tensorflow               0.0.5                
tensorflow-estimator          1.14.0               
tensorflow-hub                0.5.0                
tensorflow-metadata           0.14.0               
tensorflow-model-optimization 0.1.2                
tensorflow-probability        0.7.0                


W0816 02:08:15.801848 140323773405056 __init__.py:689] 

  TensorFlow's `tf-nightly` package will soon be updated to TensorFlow 2.0.

  Please upgrade your code to TensorFlow 2.0:
    * https://www.tensorflow.org/beta/guide/migration_guide

  Or install the latest stable TensorFlow 1.X release:
    * `pip install -U "tensorflow==1.*"`

  Otherwise your code may be broken by the change.

  


In [3]:
print(tf.__version__)

1.15.0-dev20190815


In [2]:
!pip uninstall -y tensorflow
!pip uninstall -y tf-nightly
!pip install -U tf-nightly-gpu
!pip install tensorflow-model-optimization


Uninstalling tensorflow-1.14.0:
  Successfully uninstalled tensorflow-1.14.0
     |████████████████████████████████| 410.5MB 50kB/s 
     |████████████████████████████████| 512kB 51.8MB/s 
     |████████████████████████████████| 71kB 32.4MB/s 
     |████████████████████████████████| 4.1MB 51.1MB/s 
  Created wheel for opt-einsum: filename=opt_einsum-3.0.0-cp36-none-any.whl size=58490 sha256=8572179a52ae11a9fb1f927d3aead7d4d9e3cdb7b5a3de7a67102ba4fcacb6b9
  Stored in directory: /root/.cache/pip/wheels/a1/82/aa/e6d68e36a66706a69f890a6072793d2b87730572499c626ed8
Successfully built opt-einsum
     |████████████████████████████████| 71kB 26.4MB/s 


In [0]:
%load_ext tensorboard

import tensorboard

import tensorflow as tf
tf.enable_eager_execution()
import tempfile
import zipfile
import os


In [5]:
tf.keras.backend.image_data_format()

'channels_last'

## Prepare the training data

In [0]:
batch_size = 128
num_classes = 10
epochs = 10

img_rows, img_cols = 28,28

(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()

if tf.keras.backend.image_data_format() == 'channels_first':
  x_train = x_train.reshape(x_train.shape[0],1,img_rows,img_clols)
  x_test = x_test.reshape(x_test.shape[0],1,img_rows,img_cols)
  input_shape = (1,img_rows,img_cols)
else:
  x_train = x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
  x_test = x_test.reshape(x_test.shape[0],img_rows,img_cols,1)
  input_shape = (img_rows,img_cols,1)
  

In [11]:
import numpy as np
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

x_train /= 255
x_test /= 255
print("x train shape",x_train.shape)
print("x test shape ",x_test.shape)

y_train = tf.keras.utils.to_categorical(y_train,num_classes)
y_test = tf.keras.utils.to_categorical(y_test,num_classes)


x train shape (60000, 28, 28, 1)
x test shape  (10000, 28, 28, 1)


In [0]:
#tf.keras.utils.to_categorical(y_train,num_classes)


Train a Mnist base model without pruning

In [14]:
layer = tf.keras.layers

model = tf.keras.Sequential([
    layer.Conv2D(32,5,padding='same',activation='relu',input_shape=input_shape),
    layer.MaxPooling2D((2,2),(2,2),padding='same'),
    layer.BatchNormalization(),
    layer.Conv2D(64,5,padding='same',activation='relu'),
    layer.MaxPooling2D((2,2),(2,2),padding='same'),
    layer.Flatten(),
    layer.Dense(1024,activation='relu'),
    layer.Dropout(0.4),
    layer.Dense(num_classes,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 14, 14, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              3

In [15]:
logdir = tempfile.mkdtemp()

print("Writing training logs to ",logdir)
%tensorboard --logdir={logdir}

Writing training logs to  /tmp/tmp38p90x0s


In [18]:
callbacks = [tf.keras.callbacks.TensorBoard(log_dir=logdir,profile_batch=0)]

model.compile(loss = tf.keras.losses.categorical_crossentropy,
              optimizer = 'adam',
              metrics= ['accuracy']
             )

model.fit(x_train,y_train,
          batch_size = batch_size,
          epochs = epochs,
          verbose =1,
          callbacks=callbacks,
          validation_data=(x_test,y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10


W0816 02:58:12.403311 140323773405056 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/optimizer_v2/optimizer_v2.py:466: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.


60000/60000 [==============================] - 10s 168us/sample - loss: 0.1763 - acc: 0.9512 - val_loss: 0.1121 - val_acc: 0.9840
Epoch 2/10
60000/60000 [==============================] - 4s 70us/sample - loss: 0.0454 - acc: 0.9856 - val_loss: 0.0308 - val_acc: 0.9898
Epoch 3/10
60000/60000 [==============================] - 4s 72us/sample - loss: 0.0313 - acc: 0.9902 - val_loss: 0.0354 - val_acc: 0.9877
Epoch 4/10
60000/60000 [==============================] - 4s 73us/sample - loss: 0.0236 - acc: 0.9925 - val_loss: 0.0239 - val_acc: 0.9925
Epoch 5/10
60000/60000 [==============================] - 4s 74us/sample - loss: 0.0194 - acc: 0.9938 - val_loss: 0.0309 - val_acc: 0.9909
Epoch 6/10
60000/60000 [==============================] - 4s 73us/sample - loss: 0.0181 - acc: 0.9941 - val_loss: 0.0224 - val_acc: 0.9935
Epoch 7/10
60000/60000 [==============================] - 4s 73us/sample - loss: 0.0142 - acc: 0.9955 - val_loss: 0.0281 - val_acc: 0.9914
Epoch 8/10
60000/60000 [============

In [19]:
score = model.evaluate(x_test,y_test,verbose=0)
print("test loss",score[0])
print("test accuracy",score[1])

test loss 0.03291126734791658
test accuracy 0.9915


In [20]:
#batckend agnostic way to save/restore models

_,keras_file = tempfile.mkstemp('.h5')

print("saving model to ",keras_file)

tf.keras.models.save_model(model,keras_file,include_optimizer=False)



saving model to  /tmp/tmp75mx44pr.h5


## Train a pruned MNIST model

In [23]:
import tensorflow_model_optimization
dir(tensorflow_model_optimization.sparsity.keras)

['ConstantSparsity',
 'InputLayer',
 'K',
 'PolynomialDecay',
 'PrunableLayer',
 'PruningSchedule',
 'PruningSummaries',
 'UpdatePruningStep',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'abc',
 'absolute_import',
 'callbacks',
 'constant_op',
 'custom_object_scope',
 'division',
 'dtypes',
 'keras',
 'math_ops',
 'np',
 'ops',
 'print_function',
 'prune_low_magnitude',
 'prune_scope',
 'pruning_sched',
 'pruning_wrapper',
 'six',
 'strip_pruning',
 'tf']

In [0]:
from tensorflow_model_optimization.sparsity.keras import PolynomialDecay,prune_low_magnitude

In [25]:
import numpy as np
epochs = 12
num_train_samples = x_train.shape[0]

end_step = np.ceil(1.0*num_train_samples /batch_size).astype(np.int32)*epochs

print("End Step " +str(end_step))



End Step 5628


In [32]:
pruning_params = {'pruning_schedule':PolynomialDecay(initial_sparsity=0.5,\
                                                    final_sparsity=0.9,\
                                                    begin_step = 2000,
                                                    end_step=end_step,\
                                                    frequency=100)
                 }

pruned_model = tf.keras.Sequential([
    prune_low_magnitude(layer.Conv2D(32,5,padding='same',activation='relu',\
                       input_shape=input_shape),**pruning_params),
    layer.MaxPooling2D((2,2),(2,2),padding='same'),
    layer.BatchNormalization(),
    prune_low_magnitude(layer.Conv2D(64,5,padding='same',activation='relu'),\
                       **pruning_params),
    layer.MaxPooling2D((2,2),(2,2),padding='same'),
    layer.Flatten(),
    prune_low_magnitude(layer.Dense(1024,activation='relu'),**pruning_params),
    layer.Dropout(0.4),
    prune_low_magnitude(layer.Dense(num_classes,activation='softmax'),**pruning_params)
    
])

pruned_model.summary()

W0816 03:22:49.766052 140323773405056 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:185: Layer.add_variable (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.add_weight` method instead.
W0816 03:22:50.018997 140323773405056 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_schedule.py:240: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_conv2d_2 (None, 28, 28, 32)        1634      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
prune_low_magnitude_conv2d_3 (None, 14, 14, 64)        102466    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
prune_low_magnitude_dense_2  (None, 1024)             

In [33]:
logdir = tempfile.mkdtemp()
print("writing train logs to "+logdir)

writing train logs to /tmp/tmpht_po467


In [34]:
pruned_model.compile(loss=tf.keras.losses.categorical_crossentropy,\
                     optimizer = 'adam',\
                     metrics = ['accuracy'])

callbacks = [UpdatePruningStep(),PruningSummaries(log_dir=logdir,profile_batch=0)]

pruned_model.fit(x_train,y_train,
                batch_size=batch_size,
                epochs=10,
                verbose=1,
                callbacks=callbacks,
                validation_data=(x_test,y_test))

score = pruned_model.evaluate(x_test,y_test,verbose=0)

print("test score",score[0])

print("Test accuracy",score[1])


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s 124us/sample - loss: 0.1825 - acc: 0.9496 - val_loss: 0.1017 - val_acc: 0.9811
Epoch 2/10
60000/60000 [==============================] - 6s 94us/sample - loss: 0.0487 - acc: 0.9850 - val_loss: 0.0292 - val_acc: 0.9910
Epoch 3/10
60000/60000 [==============================] - 6s 93us/sample - loss: 0.0317 - acc: 0.9901 - val_loss: 0.0274 - val_acc: 0.9906
Epoch 4/10
60000/60000 [==============================] - 6s 93us/sample - loss: 0.0246 - acc: 0.9919 - val_loss: 0.0282 - val_acc: 0.9900
Epoch 5/10
60000/60000 [==============================] - 6s 95us/sample - loss: 0.0198 - acc: 0.9940 - val_loss: 0.0228 - val_acc: 0.9920
Epoch 6/10
60000/60000 [==============================] - 6s 96us/sample - loss: 0.0161 - acc: 0.9947 - val_loss: 0.0257 - val_acc: 0.9923
Epoch 7/10
60000/60000 [==============================] - 6s 96us/sample - loss: 0.0153 - acc: 0.9951 - val_loss: 0

## Save and restor the pruned model
Continue to train for two epochs 

In [36]:

_,checkpoint_file = tempfile.mkstemp('.h5')
print("saving pruned model to",checkpoint_file)

tf.keras.models.save_model(pruned_model,checkpoint_file,include_optimizer=True)

with prune_scope():
  restore_model = tf.keras.models.load_model(checkpoint_file)
  
restore_model.fit(x_train,y_train,
                 batch_size=batch_size,
                 epochs=2,
                 verbose=1,
                 callbacks=callbacks,
                 validation_data=(x_test,y_test))

score = restore_model.evaluate(x_test,y_test,verbose=0)

print('test loss',score[0])
print("test accuracy",score[1])




saving pruned model to /tmp/tmpuxiepae1.h5


W0816 06:08:58.646966 140323773405056 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_grad.py:1428: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 7s 116us/sample - loss: 0.0112 - acc: 0.9964 - val_loss: 0.0256 - val_acc: 0.9921
Epoch 2/2
60000/60000 [==============================] - 6s 96us/sample - loss: 0.0069 - acc: 0.9977 - val_loss: 0.0253 - val_acc: 0.9927
test loss 0.02529340742647055
test accuracy 0.9927


# strip the pruning model and then export for serving

In [37]:
final_model = strip_pruning(pruned_model)
final_model.summary()

_,pruned_keras_file = tempfile.mkstemp('.h5')
print("saving pruned model to ",pruned_keras_file)

tf.keras.models.save_model(final_model,pruned_keras_file,include_optimizer=False)


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)             

In [43]:
!ls /tmp

__pycache__	tmp75mx44pr.h5	 tmpj75fs6oa.py   tmpzshmr1cv.py
tmp285hu8a_.h5	tmp8ah2ubv9.py	 tmp_pgx0xll.zip
tmp38p90x0s	tmpcziusg4d.zip  tmptij_cf15.py
tmp4q15klbo.h5	tmpht_po467	 tmpuxiepae1.h5


In [41]:
_,zip1 = tempfile.mkstemp('.zip')
with zipfile.ZipFile(zip1,'w',compression=zipfile.ZIP_DEFLATED) as f:
  f.write(keras_file)
print("size of the unpruned model before compression :%.2f Mb"%
     (os.path.getsize(keras_file)/float(2**10)))

print("size of the unpruned model after compression :%.2f Mb"%
     (os.path.getsize(zip1)/float(2**10)))

_,zip2 = tempfile.mkstemp('.zip')
with zipfile.ZipFile(zip2,'w',compression=zipfile.ZIP_DEFLATED) as f:
  f.write(pruned_keras_file)
  
  print("size of the pruned model before compression :%.2f Mb"%
       (os.path.getsize(pruned_keras_file)/float(2**10)))
  print("size of the pruned model fater compression :%.2f Mb"%
       (os.path.getsize(zip2)/float(2**10)))
  

size of the unpruned model before compression :12817.91 Mb
size of the unpruned model after compression :11865.78 Mb
size of the pruned model before compression :12818.05 Mb
size of the pruned model fater compression :2569.83 Mb


## so the pruned model after compression ,the size if 5x of the original compression model size

In [45]:
tflite_model_file = '/tmp/sparse_mnist.tflite'

converter = tf.lite.TFLiteConverter.from_keras_model_file(pruned_keras_file)

tflite_model = converter.convert()

with open(tflite_model_file,'wb') as f:
  f.write(tflite_model)
  
_,zip_tflite = tempfile.mkstemp('.zi')

with zipfile.ZipFile(zip_tflite,'w',compression=zipfile.ZIP_DEFLATED) as f:
  f.write(tflite_model_file)
print("size of the tflite model before compression : %.2f Mb"
     %(os.path.getsize(tflite_model_file)/float(2**10)))
print("size of the tflite model after compression :%.2f Mb"
     %(os.path.getsize(zip_tflite)/float(2**10)))



W0816 07:45:11.219341 140323773405056 hdf5_format.py:171] No training configuration found in save file: the model was *not* compiled. Compile it manually.


size of the tflite model before compression : 12794.81 Mb
size of the tflite model after compression :2488.21 Mb


In [50]:
import numpy as np
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]

output_index = interpreter.get_output_details()[0]["index"]

def eval_model(interperter,x_test,y_test):
  total_seen = 0
  num_correct = 0
  
  for img,label in zip(x_test,y_test):
    inp = img.reshape((1,28,28,1))
    total_seen +=1
    
    interpreter.set_tensor(input_index,inp)
    interpreter.invoke()
    
    predictions = interpreter.get_tensor(output_index)
    
    if np.argmax(predictions) == np.argmax(label):
      num_correct +=1 
    
    if total_seen %1000==0:
      print("Accuracy after %i images :%f"%
           (total_seen,float(num_correct)/float(total_seen)))
      
  return float(num_correct)/float(total_seen)
print(eval_model(interpreter,x_test,y_test))




Accuracy after 1000 images :0.992000
Accuracy after 2000 images :0.986500
Accuracy after 3000 images :0.985667
Accuracy after 4000 images :0.986500
Accuracy after 5000 images :0.986000
Accuracy after 6000 images :0.988000
Accuracy after 7000 images :0.988857
Accuracy after 8000 images :0.990000
Accuracy after 9000 images :0.990556
Accuracy after 10000 images :0.990800
0.9908


Post-trianing quantize the tensorflow lite model


In [52]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(pruned_keras_file)

converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

tflite_quant_model = converter.convert()

tflite_quant_model_file = '/tmp/sparse_mnist_quant.tflite'

with open(tflite_quant_model_file,'wb') as f:
  f.write(tflite_quant_model)
  

W0816 08:00:50.206250 140323773405056 hdf5_format.py:171] No training configuration found in save file: the model was *not* compiled. Compile it manually.


In [53]:
_,zip_tflite_quant = tempfile.mkstemp('.zip')
with zipfile.ZipFile(zip_tflite_quant,'w',compression=zipfile.ZIP_DEFLATED) as f:
  f.write(tflite_quant_model_file)
  
  print("Size of the tflite model before compression is %.2f"%
       (os.path.getsize(tflite_quant_model_file)/float(2**10)))
  print("Size of the tflite model after compression is %.2f"%
       (os.path.getsize(zip_tflite_quant)/float(2**10)))
  

Size of the tflite model before compression is 3206.89
Size of the tflite model after compression is 608.07


In [56]:
interpreter = tf.lite.Interpreter(model_path = str(tflite_quant_model_file))
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

print(eval_model(interpreter,x_test,y_test))

Accuracy after 1000 images :0.992000
Accuracy after 2000 images :0.986000
Accuracy after 3000 images :0.985333
Accuracy after 4000 images :0.986500
Accuracy after 5000 images :0.985800
Accuracy after 6000 images :0.987833
Accuracy after 7000 images :0.988714
Accuracy after 8000 images :0.989875
Accuracy after 9000 images :0.990444
Accuracy after 10000 images :0.990700
0.9907
